In [139]:
import requests
from bs4 import BeautifulSoup
import json

In [127]:
def parse_page(html):
    triplets = []
    parsed = BeautifulSoup(html.text)
    for table in parsed.find_all('table', {'class': 'table_arr2table_new_glossed_text'}):
        for word in table.find_all('table'):
            word_form = word.find_all('td')[0].text
            stem = word.find_all('td')[2].text.split('-')[0]
            tags = [word.find_all('td')[4].text.split('-')[0]]
            tags.extend(word.find_all('td')[3].text.split('-')[1:])
            triplets.append({'lemma': stem, 'word_form': word_form, 'tags': ';'.join(tags)})
    return triplets

In [128]:
def get_html(link):
    page = requests.get(url=link)
    page.encoding = 'utf=8'
    parsed = BeautifulSoup(page.text)
    triplets = []
    for text in parsed.find_all('a', {'target': '_blank'}):
        text_link = 'http://www.babel.gwi.uni-muenchen.de/' + text.attrs['href'].replace('view_corpus_file_new', 'view_glossed_corpus')
        text_page = requests.get(url=text_link)
        text_page.encoding = 'utf=8'
        triplets.extend(parse_page(text_page))
    return triplets

In [129]:
triplets = get_html('http://www.babel.gwi.uni-muenchen.de/index.php?abfrage=KK_corpus&subnavi=corpus_pub')

In [130]:
len(triplets)

83090

In [143]:
with open('website_parsed.json', 'w', encoding='utf-8') as f:
    json.dump(triplets, f, indent='\t', ensure_ascii=False)